In [1]:
import pickle
import math
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
import pickle
%matplotlib inline

import hummingbird.ml
from hummingbird.ml import constants
import torch
import lightgbm as lgb
from os.path import join
from s2and.data import ANDData

from s2and.model import PairwiseModeler
from s2and.featurizer import FeaturizationInfo, featurize
from s2and.eval import pairwise_eval

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support

from livelossplot import PlotLosses

2022-12-02 01:05:46,728 - s2and - WARNING - You haven't set `main_data_dir` in data/path_config.json! Using data/ as default data directory.


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from IPython import embed

In [4]:
wandb.login()

wandb: Currently logged in as: dhdhagar. Use `wandb login --relogin` to force relogin


True

In [3]:
def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
# Convert the S2 LGBM classifier to torch

def convert_gbdt_to_torch(classifier_model, test_input=None, dropout=0.1, 
                          fine_tune=True, force_gemm=False, 
                          fine_tune_temp={'train': 1., 'eval': 1., 'required_grad': False},
                          fine_tune_activation='tanh'):
    extra_config = {}
    
    if fine_tune:
        extra_config.update({
            constants.FINE_TUNE: True,
            constants.FINE_TUNE_DROPOUT_PROB: dropout
        })
    if force_gemm:
        extra_config[constants.TREE_IMPLEMENTATION] = "gemm"
    extra_config[constants.FINE_TUNE_TEMP] = fine_tune_temp
    extra_config[constants.FINE_TUNE_ACTIVATION] = fine_tune_activation
    
    humming = hummingbird.ml.convert(classifier_model, "torch", test_input=test_input, extra_config=extra_config)
    
    return humming.model

In [85]:
# Load S2AND production classifier and convert to NN

with open("data/production_model.pickle", "rb") as _pkl_file:  # full_union_seed_3
    prod_model = pickle.load(_pkl_file)
    if type(prod_model) == dict:
        prod_model = prod_model["clusterer"]

In [8]:
# # Compute average of AUROC and F1 for the full union models
# FEATURES_TO_USE = [
#     "name_similarity",
#     "affiliation_similarity",
#     "email_similarity",
#     "coauthor_similarity",
#     "venue_similarity",
#     "year_diff",
#     "title_similarity",
#     "reference_features",
#     "misc_features",
#     "name_counts",
#     "embedding_similarity",
#     "journal_similarity",
#     "advanced_name_similarity",
# ]

# NAMELESS_FEATURES_TO_USE = [
#     feature_name for feature_name in FEATURES_TO_USE
#     if feature_name not in {"name_similarity", "advanced_name_similarity", "name_counts"}
# ]

# auroc_all, f1_all = [], []

# for i in range(5):
#     # Load dataset

#     dataset_name = "pubmed"  # "kisti"  # "aminer"
#     parent_dir = f"data/{dataset_name}"
#     dataset = ANDData(
#         signatures=join(parent_dir, f"{dataset_name}_signatures.json"),
#         papers=join(parent_dir, f"{dataset_name}_papers.json"),
#         mode="train",
#         specter_embeddings=join(parent_dir, f"{dataset_name}_specter.pickle"),
#         clusters=join(parent_dir, f"{dataset_name}_clusters.json"),
#         block_type="s2",
#         train_pairs_size=100000,
#         val_pairs_size=10000,
#         test_pairs_size=10000,
#         name=dataset_name,
#         n_jobs=8,
#         random_seed=i+1
#     )

#     featurization_info = FeaturizationInfo()
#     FEATURIZER_INFO = FeaturizationInfo(features_to_use=FEATURES_TO_USE)
#     NAMELESS_FEATURIZER_INFO = FeaturizationInfo(features_to_use=NAMELESS_FEATURES_TO_USE)
#     # the cache will make it faster to train multiple times - it stores the features on disk for you
#     train, val, test = featurize(dataset, FEATURIZER_INFO, n_jobs=8, use_cache=True, 
#                                  nameless_featurizer_info=NAMELESS_FEATURIZER_INFO)

#     # Get x,y data for each split
#     X_train, y_train, X_train_NL = train
#     X_val, y_val, X_val_NL = val
#     X_test, y_test, X_test_NL = test
    
#     with open(f"data/full_union_seed_{i+1}.pickle", "rb") as _pkl_file:
#         prod_model = pickle.load(_pkl_file)
        
#         y_pred = prod_model.classifier.predict_proba(X_test)
#         y_prob = y_pred[:, 1]
#         y_pred_NL = prod_model.nameless_classifier.predict_proba(X_test_NL)
#         y_prob_NL = y_pred_NL[:, 1]
#         y_prob = (y_prob + y_prob_NL) / 2
#         y = y_test
#         fpr, tpr, _ = roc_curve(y, y_prob)
#         roc_auc = auc(fpr, tpr)
#         thresh_for_f1 = 0.5
#         pr, rc, f1, _ = precision_recall_fscore_support(y, y_prob > thresh_for_f1, beta=1.0, average="macro")
        
#         auroc_all.append(roc_auc)
#         f1_all.append(f1)
#         print(auroc_all)
#         print(f1_all)
#         print()
# #         print("LightGBM (AUROC, F1):", roc_auc, np.round(f1, 3))
# print("Mean AUROC:", np.mean(auroc_all))
# print("Mean F1:", np.mean(f1_all))

In [5]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using device={device}")

Using device=cuda


In [63]:
out_dir = os.path.join("data", "zbmath", "clf_random_splits")
out_fname = os.path.join(out_dir, f"splits_rand1.pkl")
with open(out_fname, 'rb') as fh:
    splits = pickle.load(fh)

In [41]:
np.sum(np.isnan(splits['X_train']))/len(splits['X_train'].flatten())

0.40719205128205127

In [136]:
np.round(np.sum(np.isnan(splits['X_train']), axis=0)/len(splits['X_train'])*100)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., 18., 18.,  0.,  0.,  0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.])

In [76]:
X_train = splits['X_train']

In [81]:
missing_per_feat = (np.sum(np.isnan(X_train), axis=0) / len(X_train))
keep_feat_mask = missing_per_feat < 0.2
X_train[:, keep_feat_mask].shape

(100000, 18)

In [133]:
X_train[:, ~keep_feat_mask] = np.zeros_like(X_train[:, ~keep_feat_mask])

In [135]:
X_train[:, ~keep_feat_mask].shape

(100000, 21)

In [119]:
hb_model = convert_gbdt_to_torch(prod_model.classifier, fine_tune=True, force_gemm=True,
                                 fine_tune_temp={'train': 1e-8, 'eval': 1e-8, 
                                                 'requires_grad': False}, 
                                 fine_tune_activation='tanh')

Finished loading model, total used 259 iterations


In [115]:
hb_model._operators[0].weight_1.data[:, ~keep_feat_mask] = torch.zeros_like(hb_model._operators[0].weight_1[:, ~keep_feat_mask])

In [130]:
not True

False

In [118]:
for p in hb_model.parameters():
    print(p.size())

torch.Size([13986, 39])
torch.Size([13986, 1])
torch.Size([259, 55, 54])
torch.Size([259, 55, 1])
torch.Size([259, 1, 55])
torch.Size([259, 1, 1])
torch.Size([])


In [126]:
torch.nn.init.zeros_(hb_model._operators[0].weight_1)

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [127]:
hb_model._operators[0].weight_1[:, ~keep_feat_mask].sum(dim=0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<SumBackward1>)

In [99]:
hb_model._operators[0].bias_1

Parameter containing:
tensor([[-1.5853e+00],
        [-3.0990e+03],
        [-1.0669e-01],
        ...,
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00]], requires_grad=True)

In [10]:
# Get data tensors and optionally convert NANs

def get_tensors(X_train, y_train, X_val, y_val, X_test, y_test, convert_nan=True, nan_val=-1):
    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)

    X_val_tensor = torch.tensor(X_val)
    y_val_tensor = torch.tensor(y_val)

    X_test_tensor = torch.tensor(X_test)
    y_test_tensor = torch.tensor(y_test)
    
    if convert_nan:
        X_train_tensor = torch.nan_to_num(X_train_tensor, nan_val)
        X_val_tensor = torch.nan_to_num(X_val_tensor, nan_val)
        X_test_tensor = torch.nan_to_num(X_test_tensor, nan_val)
    
    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [11]:
# Attempts to release memory

def release_memory(hm_model, X_train_tensor, y_train_tensor, X_val_tensor, 
                   y_val_tensor, X_test_tensor, y_test_tensor):
    hm_model = None
    X_train_tensor = None
    y_train_tensor = None
    X_val_tensor = None
    y_val_tensor = None
    X_test_tensor = None
    y_test_tensor = None
    torch.cuda.empty_cache()

In [12]:
# Load dataset
if True:
    dataset_name = "zbmath"  # "pubmed"  # "arnetminer"  # "qian"
    for dataset_random_seed in [1,2,3,4,5]:
        parent_dir = f"data/{dataset_name}"
        out_dir = os.path.join("data", dataset_name, "clf_random_splits")
        dataset = ANDData(
            signatures=join(parent_dir, f"{dataset_name}_signatures.json"),
            papers=join(parent_dir, f"{dataset_name}_papers.json"),
            mode="train",
            specter_embeddings=join(parent_dir, f"{dataset_name}_specter.pickle"),
            clusters=join(parent_dir, f"{dataset_name}_clusters.json"),
            block_type="s2",
            train_pairs_size=100000,
            val_pairs_size=10000,
            test_pairs_size=10000,
            name=dataset_name,
            n_jobs=8,
            random_seed=dataset_random_seed
        )

        featurization_info = FeaturizationInfo()
        # the cache will make it faster to train multiple times - it stores the features on disk for you
        train, val, test = featurize(dataset, featurization_info, n_jobs=8, use_cache=True)

        # Get x,y data for each split
        X_train, y_train, _ = train
        X_val, y_val, _ = val
        X_test, y_test, _ = test

        os.makedirs(out_dir, exist_ok=True)
        dict_to_save = {
            'X_train': X_train,
            'y_train': y_train,
            'X_val': X_val,
            'y_val': y_val,
            'X_test': X_test,
            'y_test': y_test
        }
        out_fname = os.path.join(out_dir, f"splits_rand{dataset_random_seed}.pkl")
        with open(out_fname, 'wb') as fh:
            pickle.dump(dict_to_save, fh, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
model, X_train_tensor, y_train_tensor, X_val_tensor, \
y_val_tensor, X_test_tensor, y_test_tensor = None, None, None, None, None, None, None

In [14]:
from neumiss import NeuMissBlock, NeuMissDEQBlock
from torch import nn

class NeuMissHB(torch.nn.Module):
    def __init__(self, n_features, neumiss_depth, hb_model):
        super().__init__()
        self.neumiss = NeuMissBlock(n_features=n_features, depth=neumiss_depth)
        self.gbdtnn = hb_model
    def forward(self, x):
        imputed = self.neumiss(x)
        return self.gbdtnn(imputed)
    
    
class NeuMissDEQHB(torch.nn.Module):
    def __init__(self, n_features, hb_model):
        super().__init__()
        self.neumiss = NeuMissDEQBlock(n_features=n_features)
        self.gbdtnn = hb_model
    def forward(self, x):
        imputed = self.neumiss(x)
        return self.gbdtnn(imputed)
    
    
class NeuMissVanilla(torch.nn.Module):
    def __init__(self, n_features, neumiss_depth, hidden_dim=1024, n_hidden_layers=10, dropout_p=0.1,
                add_neumiss=True, add_batchnorm=False):
        super().__init__()
        if add_batchnorm:
            self.linear_layer = nn.Sequential(
                *(([NeuMissBlock(n_features=n_features, depth=neumiss_depth)] if add_neumiss else []) + 
                  [nn.Linear(n_features, hidden_dim)] +
                  [nn.BatchNorm1d(hidden_dim), nn.ReLU(), nn.Dropout(p=dropout_p),
                   nn.Linear(hidden_dim, hidden_dim)] * n_hidden_layers +
                  [nn.BatchNorm1d(hidden_dim), nn.ReLU(), nn.Dropout(p=dropout_p), nn.Linear(hidden_dim, 1)])
            )
        else:
            self.linear_layer = nn.Sequential(
                *(([NeuMissBlock(n_features=n_features, depth=neumiss_depth)] if add_neumiss else []) + 
                  [nn.Linear(n_features, hidden_dim)] +
                  [nn.ReLU(), nn.Dropout(p=dropout_p),
                   nn.Linear(hidden_dim, hidden_dim)] * n_hidden_layers +
                  [nn.ReLU(), nn.Dropout(p=dropout_p), nn.Linear(hidden_dim, 1)])
            )
    def forward(self, x):
        return self.linear_layer(x)

    
class NeuMissVanillaLeaky(torch.nn.Module):
    def __init__(self, n_features, neumiss_depth, hidden_dim=1024, n_hidden_layers=10, dropout_p=0.1,
                 negative_slope=0.01, add_batchnorm=False, add_neumiss=True):
        super().__init__()
        if add_batchnorm:
            self.linear_layer = nn.Sequential(
                *(([NeuMissBlock(n_features=n_features, depth=neumiss_depth)] if add_neumiss else []) +
                  [nn.Linear(n_features, hidden_dim)] +
                  [nn.BatchNorm1d(hidden_dim), nn.LeakyReLU(negative_slope=negative_slope), nn.Dropout(p=dropout_p),
                   nn.Linear(hidden_dim, hidden_dim)] * n_hidden_layers +
                  [nn.BatchNorm1d(hidden_dim), nn.LeakyReLU(negative_slope=negative_slope), nn.Dropout(p=dropout_p), nn.Linear(hidden_dim, 1)])
            )
        else:
            self.linear_layer = nn.Sequential(
                *(([NeuMissBlock(n_features=n_features, depth=neumiss_depth)] if add_neumiss else []) +
                  [nn.Linear(n_features, hidden_dim)] +
                  [nn.LeakyReLU(negative_slope=negative_slope), nn.Dropout(p=dropout_p),
                   nn.Linear(hidden_dim, hidden_dim)] * n_hidden_layers +
                  [nn.LeakyReLU(negative_slope=negative_slope), nn.Dropout(p=dropout_p), nn.Linear(hidden_dim, 1)])
            )
    def forward(self, x):
        return self.linear_layer(x)
    

# Function to re-init weights using xavier initialization (tanh); should use He init for relu
def init_weights(model, activation="tanh", vanilla=False):
    if vanilla:
        for p in model.named_parameters():
            if 'weight' in p[0]:
                if len(p[1].data.size()) > 1:
                    if activation == "tanh":
                        torch.nn.init.xavier_uniform_(p[1].data, gain=nn.init.calculate_gain(activation))
                    else:
                        torch.nn.init.kaiming_uniform_(p[1].data, nonlinearity=activation)  # "relu" / "leaky_relu"
            elif 'bias' in p[0]:
                p[1].data.fill_(0.01)
    else:
        for p in model.parameters():
            if len(p.size()) > 0:
                if p.size(-1) == 1:
                    p.data.fill_(0.01)
                else:
                    if activation == "tanh":
                        torch.nn.init.xavier_uniform_(p.data, gain=nn.init.calculate_gain("tanh"))
                    else:
                        torch.nn.init.kaiming_uniform_(p.data, nonlinearity=activation)  # "relu" / "leaky_relu"

In [24]:
needs_sigmoid = [NeuMissVanilla, NeuMissVanillaLeaky,]
metric_to_idx = {'auroc': 0, 'f1': 1}

def predict_proba(model, input):
    if model.__class__ in needs_sigmoid:
        return torch.sigmoid(model(input.type(torch.float)))
    else:
        return model(input)[1][:, 1]
    
def evaluate(model, input, output, mode="macro", return_pred_only=False, 
             batch_size=None, overfit_one_batch=False, loss_fn=None, pos_weight=None):
    if batch_size is None:
        if model.__class__ in needs_sigmoid:
            y_prob = torch.sigmoid(model(input.type(torch.float))).cpu().numpy()
        else:
            y_prob = model(input)[1][:, 1].cpu().numpy()
    else:
        y_prob = []
        for i in range(0, batch_size if overfit_one_batch else len(input), batch_size):
            if model.__class__ in needs_sigmoid:
                _y_prob = torch.sigmoid(model(input[i:i+batch_size].type(torch.float).to(device))).cpu().numpy()
            else:
                _y_prob = model(input[i:i+batch_size].to(device))[1][:, 1].cpu().numpy()
            y_prob.append(_y_prob)
        y_prob = np.concatenate(y_prob, axis=0).flatten()
    
    if return_pred_only:
        return y_prob
    
    y = output.numpy()
    
    if batch_size is not None and overfit_one_batch:
        y = y[:batch_size]
    
    fpr, tpr, _ = roc_curve(y, y_prob)
    roc_auc = auc(fpr, tpr)
    thresh_for_f1 = 0.5
    pr, rc, f1, _ = precision_recall_fscore_support(y, y_prob > thresh_for_f1, beta=1.0, average=mode,
                                                    zero_division=0)
    
    if loss_fn is not None:
        if 'BCELoss' in loss_fn.__class__.__name__:
            if pos_weight is None:
                loss_fn.weight = None
            else:
                loss_weight = (pos_weight * torch.tensor(y).type(torch.float))
                loss_weight[loss_weight==0] = 1.
                loss_fn.weight = loss_weight
        loss = loss_fn(torch.tensor(y_prob).type(torch.float).view_as(torch.tensor(y)), torch.tensor(y).type(torch.float))
        return roc_auc, np.round(f1, 3), loss.item()
    
    return roc_auc, np.round(f1, 3)

In [22]:
def train(dataset_name="pubmed", verbose=False, hyperparams = {
    # Dataset
    "dataset": "pubmed",
    "dataset_random_seed": 1,
    # Training config
    "lr": 1e-4, 
    "n_epochs": 200,
    "weighted_loss": True,
    "batch_size": 10000,
    "use_lr_scheduler": True,
    "lr_factor": 0.6,
    "lr_min": 1e-6,
    "lr_scheduler_patience": 10,
    "weight_decay": 0.,
    "dev_opt_metric": 'auroc',  # 'f1'
    "overfit_one_batch": False,
    "convert_nan": False,
    "nan_value": -1,
    # Model config
    "hb_model": False,
    "hb_temp": 1e-8,
    "hb_activation": 'tanh',
    "neumiss_deq": False,
    "neumiss_depth": 20,
    "vanilla_hidden_dim": 1024,
    "vanilla_n_hidden_layers": 1,
    "vanilla_dropout": 0.,
    "vanilla_batchnorm": True,
    "vanilla_activation": "leaky_relu",
    "reinit_model": False
}):
#     for seed in [1,2,3,4,5]:
#         hyperparams.update({"dataset_random_seed": seed})
    # Start wandb run
    with wandb.init(config=hyperparams) as run:
        hyp = wandb.config
        
        # Load data
        out_dir = os.path.join("data", hyp["dataset"], "clf_random_splits")
        out_fname = os.path.join(out_dir, f"splits_rand{hyp['dataset_random_seed']}.pkl")
        with open(out_fname, 'rb') as fh:
            splits = pickle.load(fh)

        # Get tensors
        X_train_tensor, y_train_tensor, \
        X_val_tensor, y_val_tensor, \
        X_test_tensor, y_test_tensor = get_tensors(splits['X_train'], splits['y_train'], splits['X_val'], 
                                                   splits['y_val'], splits['X_test'], splits['y_test'], 
                                                   convert_nan=hyp['convert_nan'], nan_val=hyp['nan_value'])
        del splits

        # Create model
        if hyp['hb_model']:
            hb_model = convert_gbdt_to_torch(prod_model.classifier, fine_tune=True, force_gemm=True,
                                             fine_tune_temp={'train': hyp['hb_temp'], 'eval': hyp['hb_temp'], 
                                                             'requires_grad': False}, 
                                             fine_tune_activation=hyp['hb_activation'])
            if hyp['convert_nan']:
                model = hb_model
            else:
                if hyp['neumiss_deq']:
                    model = NeuMissDEQHB(n_features=X_train.shape[1], hb_model=hb_model)
                else:
                    model = NeuMissHB(n_features=X_train.shape[1], neumiss_depth=hyp['neumiss_depth'], 
                                      hb_model=hb_model)
        else:
            if hyp['vanilla_activation'] == 'relu':
                model = NeuMissVanilla(n_features=X_train.shape[1], neumiss_depth=hyp['neumiss_depth'], 
                                       hidden_dim=hyp['vanilla_hidden_dim'], 
                                       n_hidden_layers=hyp['vanilla_n_hidden_layers'], 
                                       dropout_p=hyp['vanilla_dropout'], add_neumiss=not hyp['convert_nan'], 
                                       add_batchnorm=hyp['vanilla_batchnorm'])
            elif hyp['vanilla_activation'] == 'leaky_relu':
                model = NeuMissVanillaLeaky(n_features=X_train.shape[1], neumiss_depth=hyp['neumiss_depth'], 
                                            hidden_dim=hyp['vanilla_hidden_dim'], 
                                            n_hidden_layers=hyp['vanilla_n_hidden_layers'], 
                                            dropout_p=hyp['vanilla_dropout'], 
                                            add_neumiss=not hyp['convert_nan'], 
                                            add_batchnorm=hyp['vanilla_batchnorm'])

        embed()
        if hyp['reinit_model']:
            if hyp['hb_model']:
                init_weights(model if hyp['convert_nan'] else model.gbdtnn, activation=hyp['hb_activation'], vanilla=False)
            else:
                init_weights(model.linear_layer, activation=hyp['vanilla_activation'], vanilla=True)
        embed()
        # Training code
        batch_size = hyp['batch_size']
        weighted_loss = hyp['weighted_loss']
        overfit_one_batch = hyp['overfit_one_batch']
        dev_opt_metric = hyp['dev_opt_metric']
        n_epochs = hyp['n_epochs']
        use_lr_scheduler = hyp['use_lr_scheduler']

        model.to(device)
        wandb.watch(model)

        y_train_tensor = y_train_tensor.float()  # Converting to keep output and prediction dtypes consistent

        pos_weight = None
        if weighted_loss:
            if overfit_one_batch:
                pos_weight = (batch_size - y_train_tensor[:batch_size].sum()) / y_train_tensor[:batch_size].sum()
            else:
                pos_weight = (len(y_train_tensor) - y_train_tensor.sum()) / y_train_tensor.sum()

        if verbose:
            print(f"Loss function pos_weight={pos_weight}")
        loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight) if model.__class__ in needs_sigmoid else \
                                                torch.nn.BCELoss()

        group_no_wd, group_wd = [], []
        for name, param in model.named_parameters():
            if '.mu' in name:
                group_no_wd.append(param)
            else:
                group_wd.append(param)
        optimizer = torch.optim.AdamW([{'params': group_wd, 'weight_decay': hyp['weight_decay']},
                                       {'params': group_no_wd, 'weight_decay': 0}], lr=hyp['lr'])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                               mode='min', 
                                                               factor=hyp['lr_factor'], 
                                                               min_lr=hyp['lr_min'],
                                                               patience=hyp['lr_scheduler_patience'], 
                                                               verbose=verbose)

        if verbose:
            print("Training:\n")
        with torch.no_grad():
            model.eval()
            init_eval_train = evaluate(model, X_train_tensor, y_train_tensor, 
                                       batch_size=batch_size, overfit_one_batch=overfit_one_batch)
            init_eval_dev = evaluate(model, X_val_tensor.to(device), y_val_tensor)
            init_eval_test = evaluate(model, X_test_tensor.to(device), y_test_tensor)

            if verbose:
                print(f"Initial model evaluation:")
                print("Train AUROC, F1:", init_eval_train)
                print("Dev AUROC, F1:", init_eval_dev)
                print("Test AUROC, F1:", init_eval_test)

            wandb.log({
                'train_auroc': init_eval_train[0],
                'train_f1': init_eval_train[1],
                'dev_auroc': init_eval_dev[0],
                'dev_f1': init_eval_dev[1],
                'test_auroc': init_eval_test[0],
                'test_f1': init_eval_test[1]})

        if verbose:
            print(f"\nDev metric to optimize: {dev_opt_metric}")

        best_model_on_dev = None
        best_metric = -1.
        best_dev_f1 = -1.
        best_dev_auroc = -1.
        best_epoch = -1

        model.train()
        loss = None

        start_time = time.time()
        for i in range(n_epochs):  # epoch
            running_loss = []
            wandb.log({'epoch': i+1})

            for j in range(0, batch_size if overfit_one_batch else len(X_train_tensor), batch_size):
                X_batch = X_train_tensor[j:j+batch_size].to(device)
                y_batch = y_train_tensor[j:j+batch_size].to(device)

                optimizer.zero_grad()
                y_ = predict_proba(model, X_batch)
                assert y_.requires_grad

                if weighted_loss and 'BCELoss' in loss_fn.__class__.__name__:
                    weights = (pos_weight * y_batch)
                    weights[weights==0] = 1.
                    loss_fn.weight = weights
                loss = loss_fn(y_.view_as(y_batch), y_batch)
                running_loss.append(loss.item())
                loss.backward()
                optimizer.step()

                # Print batch loss
                if verbose:
                    print("\tBatch", f"[{j}:{j+batch_size}]", ":", running_loss[-1])
                wandb.log({'train_loss_batch': running_loss[-1]})

            # Print epoch validation accuracy
            with torch.no_grad():
                model.eval()
                dev_auroc_f1_loss = evaluate(model, X_val_tensor.to(device), y_val_tensor, 
                                             loss_fn=loss_fn, pos_weight=pos_weight)
                if verbose:
                    print("Epoch", i+1, ":", "Dev AUROC,F1,loss:", dev_auroc_f1_loss)
                if dev_auroc_f1_loss[metric_to_idx[dev_opt_metric]] > best_metric:
                    if verbose:
                        print(f"New best dev {dev_opt_metric}; storing model")
                    best_epoch = i
                    best_metric = dev_auroc_f1_loss[metric_to_idx[dev_opt_metric]]
                    best_dev_f1 = dev_auroc_f1_loss[1]
                    best_dev_auroc = dev_auroc_f1_loss[0]
                    best_model_on_dev = copy.deepcopy(model)
                if overfit_one_batch:
                    train_auroc_f1 = evaluate(model, X_batch.to(device), y_batch.cpu())
            model.train()

            wandb.log({
                'train_loss_epoch': np.mean(running_loss),
                'dev_auroc': dev_auroc_f1_loss[0],
                'dev_f1': dev_auroc_f1_loss[1],
                'dev_loss': dev_auroc_f1_loss[2],
            })
            if overfit_one_batch:
                wandb.log({'train_auroc': train_auroc_f1[0], 
                           'train_f1': train_auroc_f1[1]})

            # Update lr schedule
            if use_lr_scheduler:
                scheduler.step(dev_auroc_f1_loss[2])  # running_loss

        end_time = time.time()

        with torch.no_grad():
            best_model_on_dev.eval()
            if verbose:
                print("----------------")
                print(f"Initial model evaluation:")
                print("Train AUROC, F1:", init_eval_train)
                print("Dev AUROC, F1:", init_eval_dev)
                print("Test AUROC, F1:", init_eval_test)
                print()

            best_eval_train = evaluate(best_model_on_dev, X_train_tensor, y_train_tensor, 
                                       batch_size=batch_size, overfit_one_batch=overfit_one_batch)
            best_eval_dev = (best_dev_auroc, best_dev_f1)
            best_eval_test = evaluate(best_model_on_dev, X_test_tensor.to(device), y_test_tensor)
            if verbose:
                print(f"Best dev eval on Epoch {best_epoch}:")
                print("Train AUROC, F1:", best_eval_train)
                print("Dev AUROC, F1:", best_eval_dev)
                print("Test AUROC, F1:", best_eval_test)
                print(f"Time taken: {end_time - start_time}")
                print()
            wandb.log({
                'best_train_auroc': best_eval_train[0],
                'best_train_f1': best_eval_train[1],
                'best_dev_auroc': best_eval_dev[0],
                'best_dev_f1': best_eval_dev[1],
                'best_test_auroc': best_eval_test[0],
                'best_test_f1': best_eval_test[1]
            })

            model.eval()
            final_eval_train = evaluate(model, X_train_tensor, y_train_tensor, 
                                        batch_size=batch_size, overfit_one_batch=overfit_one_batch)
            final_eval_dev = evaluate(model, X_val_tensor.to(device), y_val_tensor)
            final_eval_test = evaluate(model, X_test_tensor.to(device), y_test_tensor)
            if verbose:
                print(f"Final model eval on Epoch {n_epochs}:")
                print("Train AUROC, F1:", final_eval_train)
                print("Dev AUROC, F1:", final_eval_dev)
                print("Test AUROC, F1:", final_eval_test)
            wandb.log({
                'train_auroc': final_eval_train[0],
                'train_f1': final_eval_train[1],
                'dev_auroc': final_eval_dev[0],
                'dev_f1': final_eval_dev[1],
                'test_auroc': final_eval_test[0],
                'test_f1': final_eval_test[1]
            })

In [23]:
train(hyperparams={
    # Dataset
    "dataset": "pubmed",
    "dataset_random_seed": 1,
    # Training config
    "lr": 2e-4, 
    "n_epochs": 200,
    "weighted_loss": False,
    "batch_size": 10000,
    "use_lr_scheduler": True,
    "lr_factor": 0.6,
    "lr_min": 1e-6,
    "lr_scheduler_patience": 10,
    "weight_decay": 0.01,
    "dev_opt_metric": 'auroc',
    "overfit_one_batch": False,
    "convert_nan": True,
    "nan_value": 0,
    # Model config
    "hb_model": True,
    "hb_temp": 1e-8,
    "hb_activation": 'tanh',
    "neumiss_deq": False,
    "neumiss_depth": 20,
    "vanilla_hidden_dim": 1024,
    "vanilla_n_hidden_layers": 1,
    "vanilla_dropout": 0.,
    "vanilla_batchnorm": True,
    "vanilla_activation": "leaky_relu",
    "reinit_model": True
})

Finished loading model, total used 259 iterations
Python 3.7.0 (default, Oct  9 2018, 10:31:47) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.31.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: exit

Python 3.7.0 (default, Oct  9 2018, 10:31:47) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.31.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: model
Out[1]: 
Executor(
  (_operators): ModuleList(
    (0): GEMMGBDTImplTraining(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [2]: model._operators[0]
Out[2]: 
GEMMGBDTImplTraining(
  (dropout): Dropout(p=0.1, inplace=False)
)

In [3]: model._operators[0].weight_1
Out[3]: 
Parameter containing:
tensor([[-0.0189,  0.0061,  0.0325,  ..., -0.0050, -0.0257,  0.0047],
        [-0.0335, -0.0151, -0.0013,  ..., -0.0271, -0.0177, -0.0278],
        [-0.0243, -0.0187,  0.0304,  ..., -0.0040,  0.0078, -0.0155],
        ...,
        [-0.0247, -0.0001, -0.

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


best_dev_auroc,▁
best_dev_f1,▁
best_test_auroc,▁
best_test_f1,▁
best_train_auroc,▁
best_train_f1,▁
dev_auroc,▁▅▅▆▇▇█▇█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
dev_f1,▁████████████▇██████████████████████████
dev_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_auroc,▁█


In [ ]:
dataset_name = "pubmed"

sweep_config = {
    'method': 'bayes',
    'name': 'vanilla_neumiss',  
    'metric': {
        'goal': 'maximize', 
        'name': 'dev_auroc'
    },
    'parameters': {
        # Training
        'lr': {'max': 1e-2, 'min': 1e-6},
        'weighted_loss': {'values': [True, False]},
        'weight_decay': {'values': [1e-1, 1e-2, 1e-3, 0]},
        'dev_opt_metric': {'values': ['auroc', 'f1']},
        "convert_nan": {'value': False},
        "nan_value": {'value': -1},
        # Model
        "hb_model": {'value': False},
#         "hb_temp": {'values': [1., 1e-1, 1e-2, 1e-4, 1e-6, 1e-8]},
#         "hb_activation": {'value': 'tanh'},
        "neumiss_deq": {'value': False},
        "neumiss_depth": {'values': [5, 10, 20]},
        "vanilla_hidden_dim": {'values': [128, 256, 512, 1024, 2048]},
        "vanilla_n_hidden_layers": {'values': [1, 2, 3]},
        "vanilla_dropout": {'values': [0., 0.1, 0.2, 0.4]},
        "vanilla_batchnorm": {'values': [True, False]},
        "reinit_model": {'values': [False, True]}
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 40
    }
}       

# Init sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="missing-values", entity="dhdhagar")
# Start sweep job
wandb.agent(sweep_id, function=lambda: train(dataset_name=dataset_name))

Create sweep with ID: rsnwx2s5
Sweep URL: https://wandb.ai/dhdhagar/missing-values/sweeps/rsnwx2s5


wandb: Agent Starting Run: hiirn3rs with config:
wandb: 	convert_nan: False
wandb: 	dev_opt_metric: f1
wandb: 	hb_model: False
wandb: 	lr: 0.00032670294650931494
wandb: 	nan_value: -1
wandb: 	neumiss_depth: 5
wandb: 	neumiss_deq: False
wandb: 	reinit_model: False
wandb: 	vanilla_batchnorm: True
wandb: 	vanilla_dropout: 0.2
wandb: 	vanilla_hidden_dim: 512
wandb: 	vanilla_n_hidden_layers: 1
wandb: 	weight_decay: 0.1
wandb: 	weighted_loss: False


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


best_dev_auroc,▁
best_dev_f1,▁
best_test_auroc,▁
best_test_f1,▁
best_train_auroc,▁
best_train_f1,▁
dev_auroc,▂▁▆▆▆▅▆▆▆▆▇▇▆▇▇▇▇███▇▇▇▇▇███████████████
dev_f1,▁▃▆▃▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇██▇▇▇▇▇█████████
dev_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_auroc,▁█


wandb: Agent Starting Run: xehu2gwp with config:
wandb: 	convert_nan: False
wandb: 	dev_opt_metric: f1
wandb: 	hb_model: False
wandb: 	lr: 0.008108765634013153
wandb: 	nan_value: -1
wandb: 	neumiss_depth: 5
wandb: 	neumiss_deq: False
wandb: 	reinit_model: True
wandb: 	vanilla_batchnorm: True
wandb: 	vanilla_dropout: 0.4
wandb: 	vanilla_hidden_dim: 256
wandb: 	vanilla_n_hidden_layers: 1
wandb: 	weight_decay: 0
wandb: 	weighted_loss: True


...

wandb: Agent Starting Run: tlp5255r with config:
wandb: 	convert_nan: False
wandb: 	dev_opt_metric: f1
wandb: 	hb_model: False
wandb: 	lr: 0.003244107010490045
wandb: 	nan_value: -1
wandb: 	neumiss_depth: 20
wandb: 	neumiss_deq: False
wandb: 	reinit_model: False
wandb: 	vanilla_batchnorm: False
wandb: 	vanilla_dropout: 0.4
wandb: 	vanilla_hidden_dim: 512
wandb: 	vanilla_n_hidden_layers: 2
wandb: 	weight_decay: 0.1
wandb: 	weighted_loss: False


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


best_dev_auroc,▁
best_dev_f1,▁
best_test_auroc,▁
best_test_f1,▁
best_train_auroc,▁
best_train_f1,▁
dev_auroc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dev_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dev_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_auroc,▁█


wandb: Agent Starting Run: ux2nrn3t with config:
wandb: 	convert_nan: False
wandb: 	dev_opt_metric: f1
wandb: 	hb_model: False
wandb: 	lr: 0.004497638378730614
wandb: 	nan_value: -1
wandb: 	neumiss_depth: 5
wandb: 	neumiss_deq: False
wandb: 	reinit_model: True
wandb: 	vanilla_batchnorm: False
wandb: 	vanilla_dropout: 0.1
wandb: 	vanilla_hidden_dim: 128
wandb: 	vanilla_n_hidden_layers: 2
wandb: 	weight_decay: 0
wandb: 	weighted_loss: False


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


## Sweeps TODO:
- vanilla_neumiss_deq
- vanilla_constant_0
- vanilla_constant_-1
- hb_neumiss
- hb_neumiss_deq
- hb_constant_0
- hb_constant_-1

In [ ]:
hm_model.to(device)
with torch.no_grad():
    hm_model.eval()
    print("----------------")
    print("Train AUROC, F1:", evaluate(hm_model, X_train_tensor, y_train_tensor, 
                                       batch_size=batch_size, overfit_one_batch=overfit_one_batch))
    print("Dev AUROC, F1:", evaluate(hm_model, X_val_tensor.to(device), y_val_tensor))
    print("Test AUROC, F1:", evaluate(hm_model, X_test_tensor.to(device), y_test_tensor))
    print("----------------")

In [ ]:
"""
Hyperparams tried:

- wd=0, lr=1e-3

- wd=1e-6, lr=1e-4
"""

In [ ]:
# hummingbird model with no imputation
# All nan's

# hummingbird model (w/o FINE_TUNING) with no imputation
# Test AUROC,F1: (0.5001742767514814, 0.123)

In [ ]:
# -------
# WITH FINE_TUNING=TRUE:

# hummingbird model trained with -1-imputation
# Test AUROC,F1: (0.8209078054716143, 0.719)

# hummingbird model with -1-imputation (no training)
# Test AUROC,F1: (0.7352596440016811, 0.522)

# -------
# WITH FINE_TUNING=FALSE:

# hummingbird model (w/o FINE_TUNING) with -1-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

# -------
# WITH TEMPERATURE=1e-8:

# hummingbird model with -1-imputation (no training)
# Test AUROC, F1: (0.9002576493481701 0.766)

# hummingbird model trained with -1-imputation
# Test AUROC, F1: (0.9735705699070798, 0.892)

In [ ]:
# hummingbird model trained with 0-imputation
# Test AUROC,F1: (0.8258480238334197, 0.718)

# hummingbird model with 0-imputation (no training)
# Test AUROC,F1: (0.7909741574142352, 0.531)

# hummingbird model (w/o FINE_TUNING) with 0-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

# -------
# WITH TEMPERATURE=1e-8:

# hummingbird model with 0-imputation (no training)
# Test AUROC, F1: (0.9468537228791971, 0.86)

# hummingbird model trained with 0-imputation
# Test AUROC, F1: (0.9766426615975862, 0.884)

In [ ]:
# hummingbird model trained with -10000-imputation
# Test AUROC,F1: (0.8402508150638835, 0.684)

# hummingbird model with -10000-imputation (no training)
# Test AUROC,F1: (0.7339247733297103, 0.576)

# hummingbird model (w/o FINE_TUNING) with -10000-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

# -------
# WITH TEMPERATURE=1e-8:

# hummingbird model with -10000-imputation (no training)
# Test AUROC, F1: (0.9002576493481701, 0.766)

# hummingbird model trained with -10000-imputation
# Test AUROC, F1: (0.9703568548980691, 0.891)

In [ ]:
# lightgm model with no imputation (i.e. with missing values)
# Test AUROC, F1: (0.9906858063325308, 0.934)

In [ ]:
# lightgm model with -1-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

In [ ]:
# lightgm model with 0-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

In [ ]:
# lightgm model with -10000-imputation
# Test AUROC, F1: (0.9277387451790313, 0.788)

In [ ]:
# lightgm model with +10000-imputation
# Test AUROC, F1: (0.8665702326967875, 0.685)

In [195]:
prod_model.classifier

LGBMClassifier(colsample_bytree=0.9476923894333807,
               learning_rate=0.09140456060593606, max_depth=45, metric='auc',
               min_child_samples=1779, min_child_weight=1.4729203134152992e-06,
               min_split_gain=1.184108047915407,
               monotone_constraints='1,1,1,0,0,0,0,1,1,1,0,1,0,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,-1,-1,-1,0,-1,0,0,0,0,0,0',
               monotone_constraints_method='advanced',
               monotone_penalty=1.009268715649836, n_estimators=1796, n_jobs=16,
               num_leaves=55, objective='binary', random_state=42,
               reg_alpha=7.44123492799962e-05,
               reg_lambda=1.8145145831930983e-05, subsample=0.7930734824387159,
               tree_learner='data', verbose=-1)

In [147]:
# LightGBM classification result on the Test set
# indices = torch.randint(0,10000,(2000,))

y_pred = prod_model.classifier.predict_proba(X_test_tensor.cpu().numpy())  # ['clusterer']
y_prob = y_pred[:, 1]
y = y_test
fpr, tpr, _ = roc_curve(y, y_prob)
roc_auc = auc(fpr, tpr)
thresh_for_f1 = 0.5
pr, rc, f1, _ = precision_recall_fscore_support(y, y_prob > thresh_for_f1, beta=1.0, average="macro")
print("LightGBM (AUROC, F1):", roc_auc, np.round(f1, 3))

LightGBM (AUROC, F1): 0.9906858063325308 0.934


In [111]:
# Hummingbird-converted model without any further fine-tuning and constant imputation
hm_model.eval()
hm_model.to(device)
with torch.no_grad():
    y_pred = hm_model(X_test_tensor.to(device))

y_prob = y_pred[1][:,1].cpu().numpy()
y = y_test
fpr, tpr, _ = roc_curve(y, y_prob)
roc_auc = auc(fpr, tpr)
thresh_for_f1 = 0.5
pr, rc, f1, _ = precision_recall_fscore_support(y, y_prob > thresh_for_f1, beta=1.0, average="macro")
print("Hummingbird test (AUROC, F1):", roc_auc, np.round(f1, 3))

with torch.no_grad():
    y_pred = hm_model(X_val_tensor.to(device))
y_prob = y_pred[1][:,1].cpu().numpy()
y = y_val
fpr, tpr, _ = roc_curve(y, y_prob)
roc_auc = auc(fpr, tpr)
thresh_for_f1 = 0.5
pr, rc, f1, _ = precision_recall_fscore_support(y, y_prob > thresh_for_f1, beta=1.0, average="macro")
print("Hummingbird dev (AUROC, F1):", roc_auc, np.round(f1, 3))

Hummingbird test (AUROC, F1): 0.4521736348042731 0.078
Hummingbird dev (AUROC, F1): 0.5465048542422006 0.085


In [18]:
X_test_tensor.shape

torch.Size([10000, 39])

In [ ]:
# import inspect
# print(inspect.getsource(hm_model._operators[0].aggregate))